# A brief introduction to the patent system
The patent is a register, tipically a document, that to document a exclusive discovery, invention or method and aims to give to the patent holder exclusive rights over the discovery/invention.

<--TODO: Explain the T,O method of organizing patents -->

To organize the patents and find a suitable way to structure its information, a commonly used method defines a patent with 2 characteristics:
1. **Task:** the method used in the described patent. In can be compress something or agilize a effect, for example.
2. **Object:** the "target" of the task. It can be a food, a construction material or any other object that, combined with the task, defines the patent.

This method is defined by the Hallbach matrix, that defines a list of Task and Objects that can be extracted from the Title or the Resume of the patent.

# T,O Finder
The T,O Finder is the method that identifies the Task and the Object from a given patent and in this notebook we will construct a method to do such thing.

In [7]:
import pandas as pd
import os
from dotenv import load_dotenv

from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

load_dotenv()

True

In [2]:
df_triz = pd.read_csv("../../data/processed/base_efeitos_físicos_publicada_lemmatized.csv")
df_triz.head()

,TIPO DE EFEITO,TAREFA,OBJETO,EFEITO FÍSICO,SINONIMO 1 EFEITO FISICO,SINONIMO 2 EFEITO FISICO,PT Link,PT Description,Link Wiki (English),TAREFA_lemmatized
0,Aplicação,Apertar,Sólido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,apertar
1,Aplicação,Apertar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,apertar
2,Aplicação,Concentrar,Campo,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,concentrar
3,Aplicação,Concentrar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,concentrar
4,Aplicação,Depositar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,depositar


In [3]:
# Cria os pares (conteúdo textual + metadados)
def build_entry(row):
    text = f'O "{row["TAREFA"]}" é um {row["TIPO DE EFEITO"]}, que no {row["OBJETO"]} causa {row["EFEITO FÍSICO"]}.'
    metadata = {
        "tipo_de_efeito": row["TIPO DE EFEITO"],
        "tarefa": row["TAREFA"],
        "objeto": row["OBJETO"],
        "efeito_fisico": row["EFEITO FÍSICO"]
    }
    return {"text": text, "metadata": metadata}

documents = [build_entry(row) for _, row in df_triz.iterrows()]
documents[:2]

[{'text': 'O "Apertar" é um Aplicação, que no Sólido causa \xa0Matriz de Halbach.',
  'metadata': {'tipo_de_efeito': 'Aplicação',
   'tarefa': 'Apertar',
   'objeto': 'Sólido',
   'efeito_fisico': '\xa0Matriz de Halbach'}},
 {'text': 'O "Apertar" é um Aplicação, que no Sólido dividido causa \xa0Matriz de Halbach.',
  'metadata': {'tipo_de_efeito': 'Aplicação',
   'tarefa': 'Apertar',
   'objeto': 'Sólido dividido',
   'efeito_fisico': '\xa0Matriz de Halbach'}}]

In [4]:
# Criar os objetos Document do LangChain
docs = [
    Document(page_content=item["text"], metadata=item["metadata"])
    for item in documents
]

# Usar embeddings HuggingFace
embedding_model = OpenAIEmbeddings()

# Criar diretório temporário para o Chroma
persist_directory = "chroma_rag_tabular"

# Criar o vectorstore Chroma
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory=persist_directory
)

# Persistir a base
vectorstore.persist()

persist_directory  # Mostrar o caminho onde foi salvo

/tmp/ipykernel_37658/2618466962.py:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()
/tmp/ipykernel_37658/2618466962.py:21: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


'chroma_rag_tabular'

In [5]:
from langchain.vectorstores import Chroma

# Reabre o banco vetorial
vectorstore = Chroma(
    persist_directory="chroma_rag_tabular",
    embedding_function=embedding_model
)

# Busca por similaridade
query = "Qual tarefa pode causar aumento da temperatura?"
results = vectorstore.similarity_search(query, k=2)

for doc in results:
    print(doc.page_content)
    print("Metadados:", doc.metadata)


/tmp/ipykernel_37658/1575818904.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


O "Corroer" é um Efeito, que no Sólido causa Hipertermia.
Metadados: {'objeto': 'Sólido', 'tarefa': 'Corroer', 'tipo_de_efeito': 'Efeito', 'efeito_fisico': 'Hipertermia'}
O "Aquecer" é um Efeito, que no Líquido causa Dilatação térmica.
Metadados: {'tipo_de_efeito': 'Efeito', 'tarefa': 'Aquecer', 'objeto': 'Líquido', 'efeito_fisico': 'Dilatação térmica'}


In [14]:
# 💬 Modelo LLM
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

# Build prompt
template = """A seguir, você receberá o título e o abstract de uma patente em português.
Use o contexto oriundo da base TRIZ de objeto, tarefa, tipo de efeito e efeito físico a 
seguir para conectar a patente a uma relação na TRIZ. Se você não conseguir fazer uma resposta com o 
contexto, apenas diga que não sabe, não tente inventar uma resposta.
Contexto: {context}
{question}
Resposta:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    # memory=memory
)




In [9]:
df_patents = pd.read_csv("../../data/processed/patentes_inpi_english_matched.csv")
df_patents.head()

,id_pedido,data_deposito,titulo,ipc,url,resumo,classifica_ipc,titulo_english,match_top_10_title
0,BR 11 2021 018393 0,02/03/2020,TRATAMENTO DE COLISÕES EM UPLINK,H04L 1/18,https://busca.inpi.gov.br/pePI/servlet/Patente...,"A presente invenção se refere a métodos, sis...",H04L 1/18,Treatment of collisions in Uplink,"{'Move', 'Break Down', 'Change Phase', 'Separa..."
1,BR 11 2021 018071 0,02/03/2020,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO ANT...,H01T 13/14,https://busca.inpi.gov.br/pePI/servlet/Patente...,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO A...,H01T 13/14 ; H01T 13/20 ; H01T 13/32 ; H0...,"In this case, it is necessary to ensure that y...","{'Move', 'Break Down'}"
2,BR 11 2021 016947 4,02/03/2020,ANTICORPOS QUE RECONHECEM TAU,C07K 16/18,https://busca.inpi.gov.br/pePI/servlet/Patente...,ANTICORPOS QUE RECONHECEM TAU. A invenção fo...,C07K 16/18 ; G01N 33/68,Antibodies that recognize you,"{'Move', 'Break Down', 'Change Phase', 'Cool',..."
3,BR 10 2020 004169 0,02/03/2020,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTÃO PAR...,F24H 3/00,https://busca.inpi.gov.br/pePI/servlet/Patente...,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTAO P...,F24H 3/008 ; F24H 4/06,Air heater with double exhaust to be used in a...,"{'Move', 'Break Down', 'Expand', 'Separate', '..."
4,BR 11 2021 006234 3,02/03/2020,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNICOS...,C12N 15/10,https://busca.inpi.gov.br/pePI/servlet/Patente...,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNIC...,C12N 15/10,Unique cell libraries and unique high-end nucl...,"{'Remove', 'Break Down', 'Move', 'Concentrate'..."


In [15]:
i = 0
title = df_patents.iloc[i]["titulo"]
abstract = df_patents.iloc[i]["resumo"]

query = f"""Título: {title}
Abstract: {abstract}"""


result = qa_chain({"query": query})

print(result["result"])
print('\n')

A patente "TRATAMENTO DE COLISÕES EM UPLINK" pode ser conectada ao efeito "Juntar" na TRIZ. Isso porque a invenção trata de juntar ou resolver colisões entre vários canais sobrepostos em comunicações sem fio, o que pode ser visto como uma forma de "juntar" canais que estão colidindo. O efeito físico causado no campo seria a Parallel Junction, que pode ser interpretada como a junção paralela ou simultânea de vários canais de comunicação.


